In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import os 
import shutil

import contact_map_generator_from_shortest_path as cmgsp
from contact_map_generator_from_shortest_path import contact_map_generator_from_SMC_list_and_DSBs

import sys
sys.setrecursionlimit(4000)

from multiprocessing import Pool
import pickle

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

from itertools import product

from scipy.ndimage.filters import gaussian_filter
import matplotlib.colors as clr 

import timeit
from scipy.sparse import coo_matrix


from cooltools.lib.numutils import iterative_correction_symmetric

## Get SMC positions data

In [ ]:
base_folder_string = 'output_0310_tiling600_2populationNonBypassingLEFs_GapMethod_SparseDSB_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('.') if base_folder_string in foldername]

# Parameter combination giving more than 95% synapsis efficiency
par_combo = ['sep125_proc250_superportion0.2_superproc5000_ctcf16_dsb4_superloading1000_bs0.5']

time_point_list = [0]
box_size = 2500 # number of monomers on each side of the DSB that we will make the contact map for
short_list_beforeDSB = []
for m in range(len(folders)):
    for n in range(len(par_combo)):
        data_dir = folders[m]
        par = par_combo[n]
        smc_file = data_dir + '/LEFcoordinates_' + par + '.npy'
        SMCs = np.load(smc_file)

        dsb_file = data_dir +'/DSB_boundary_coordinates.npy'
        with open(dsb_file, 'rb') as infile:
            DSBs = np.load(infile)
            BEs = np.load(infile)

        smcs_dict = {}
        for t in time_point_list:
            smcs_dict[t] = [(int(l),int(r)) for (l,r) in zip(SMCs[2*t,:],SMCs[2*t+1,:])]

        for d in DSBs[1::2]: # d is the right DSB end
            
            shift_factor = d-box_size

            for t in time_point_list: #smcs_dict.keys():
                smcs = [(x[0]-shift_factor,x[1]-shift_factor) for x in smcs_dict[t] if (np.abs(x[0]-d)<=box_size)  and (np.abs(x[1]-d)<box_size)]
                #short_list.append(smcs)
                has_constraint = False
                for x in smcs:
                    if x[0]<box_size and x[1]>box_size:
                        has_constraint = True
                        break
                if has_constraint==True:
                    short_list_beforeDSB.append(smcs) 

In [ ]:
base = 50
nreps = 2
for si, smcs in enumerate(short_list_beforeDSB):
    dsb = [box_size-1,box_size]
    L_tot = 2*box_size + base
    
    for rep in range(nreps):

        m_samples1000_beforeDSB, c_samples1000_beforeDSB = contact_map_generator_from_SMC_list_and_DSBs(smcs,dsb,L_tot,base=base,nsamples=1000)

        if rep == 0 and si == 0:
            M_samples1000_beforeDSB = m_samples1000_beforeDSB
            C_samples1000_beforeDSB = c_samples1000_beforeDSB
        else:
            M_samples1000_beforeDSB += m_samples1000_beforeDSB
            C_samples1000_beforeDSB += c_samples1000_beforeDSB
        

In [ ]:
M_new_samples1000_beforeDSB = np.zeros((100,100))
C_new_samples1000_beforeDSB = np.zeros((100,100))
for x in range(100):
    for y in range(100):
        M_new_samples1000_beforeDSB[x//1,y//1] += M_samples1000_beforeDSB[x,y]
        C_new_samples1000_beforeDSB[x//1,y//1] += C_samples1000_beforeDSB[x,y]

In [ ]:
base_folder_string = 'output_0310_tiling600_2populationNonBypassingLEFs_GapMethod_SparseDSB_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('.') if base_folder_string in foldername]

# Parameter combination giving more than 95% synapsis efficiency
par_combo = ['sep125_proc250_superportion0.2_superproc5000_ctcf16_dsb4_superloading1000_bs0.5']

time_point_list = [6]
box_size = 2500 # number of monomers on each side of the DSB that we will make the contact map for
short_list = []
for m in range(len(folders)):
    for n in range(len(par_combo)):
        data_dir = folders[m]
        par = par_combo[n]
        smc_file = data_dir + '/LEFcoordinates_' + par + '.npy'
        SMCs = np.load(smc_file)

        dsb_file = data_dir +'/DSB_boundary_coordinates.npy'
        with open(dsb_file, 'rb') as infile:
            DSBs = np.load(infile)
            BEs = np.load(infile)

        smcs_dict = {}
        for t in time_point_list:
            smcs_dict[t] = [(int(l),int(r)) for (l,r) in zip(SMCs[2*t,:],SMCs[2*t+1,:])]

        for d in DSBs[1::2]: # d is the right DSB end

            shift_factor = d-box_size

            for t in time_point_list: #smcs_dict.keys():
                smcs = [(x[0]-shift_factor,x[1]-shift_factor) for x in smcs_dict[t] if (np.abs(x[0]-d)<=box_size)  and (np.abs(x[1]-d)<box_size)]

                has_constraint = False
                for x in smcs:
                    if x[0]<box_size and x[1]>box_size:
                        has_constraint = True
                        break
                if has_constraint==True:
                    short_list.append(smcs) 

In [ ]:
# number of DSBs in the contact map
len(short_list)

In [ ]:
base = 50
nreps = 2
for si, smcs in enumerate(short_list):
    dsb = [box_size-1,box_size] 
    L_tot = 2*box_size + base
    
    for rep in range(nreps):
        m_samples1000, c_samples1000 = contact_map_generator_from_SMC_list_and_DSBs(smcs,dsb,L_tot,base=base,nsamples=1000)
        if rep == 0 and si == 0:
            M_samples1000_120min = m_samples1000
            C_samples1000_120min = c_samples1000
        else:
            M_samples1000_120min += m_samples1000
            C_samples1000_120min += c_samples1000
        

In [ ]:
M_new_samples1000 = np.zeros((100,100))
C_new_samples1000 = np.zeros((100,100))
for x in range(100):
    for y in range(100):
        M_new_samples1000[x//1,y//1] += M_samples1000_120min[x,y]
        C_new_samples1000[x//1,y//1] += C_samples1000_120min[x,y]

In [ ]:
raw=(M_new_samples1000/C_new_samples1000)/(M_new_samples1000_beforeDSB/C_new_samples1000_beforeDSB)
corrected=iterative_correction_symmetric(raw)

In [ ]:
plt.imshow(np.log2(corrected[0]),cmap='cividis',vmax=0.4,vmin=-0.4)
# plt.hlines(50,0,100,linestyles=':')
# plt.vlines(50,0,100,linestyles=':')
plt.xlim([0,100])
plt.ylim([0,100])

plt.colorbar() 
plt.title("With DSB")
plt.savefig('Figures/'+'HiC_120min_2rep_1000samples_DividedByBeforeBSB_allTADs_vmax0.4_vmin-0.4_3rdParameterCombo_IterativeCorrected.pdf',format='pdf',bbox_inches='tight')

In [ ]:
base_folder_string = 'output_0310_tiling600_2populationNonBypassingLEFs_GapMethod_SparseDSB_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('.') if base_folder_string in foldername]

# Parameter combination giving more than 95% synapsis efficiency
par_combo = ['sep125_proc250_superportion0.2_superproc5000_ctcf16_dsb4_superloading1000_bs0.5']

time_point_list = [4]
box_size = 2500 # number of monomers on each side of the DSB that we will make the contact map for
short_list = []
for m in range(len(folders)):
    for n in range(len(par_combo)):
        data_dir = folders[m]
        par = par_combo[n]
        smc_file = data_dir + '/LEFcoordinates_' + par + '.npy'
        SMCs = np.load(smc_file)

        dsb_file = data_dir +'/DSB_boundary_coordinates.npy'
        with open(dsb_file, 'rb') as infile:
            DSBs = np.load(infile)
            BEs = np.load(infile)

        smcs_dict = {}
        for t in time_point_list:
            smcs_dict[t] = [(int(l),int(r)) for (l,r) in zip(SMCs[2*t,:],SMCs[2*t+1,:])]

        for d in DSBs[1::2]: # d is the right DSB end

            shift_factor = d-box_size

            for t in time_point_list: #smcs_dict.keys():
                smcs = [(x[0]-shift_factor,x[1]-shift_factor) for x in smcs_dict[t] if (np.abs(x[0]-d)<=box_size)  and (np.abs(x[1]-d)<box_size)]

                has_constraint = False
                for x in smcs:
                    if x[0]<box_size and x[1]>box_size:
                        has_constraint = True
                        break
                if has_constraint==True:
                    short_list.append(smcs) 

In [ ]:
base = 50
nreps = 2
for si, smcs in enumerate(short_list):
    dsb = [box_size-1,box_size] 
    L_tot = 2*box_size + base
    
    for rep in range(nreps):

        m_samples1000, c_samples1000 = contact_map_generator_from_SMC_list_and_DSBs(smcs,dsb,L_tot,base=base,nsamples=1000)

        if rep == 0 and si == 0:
            M_samples1000_60min = m_samples1000
            C_samples1000_60min = c_samples1000
        else:
            M_samples1000_60min += m_samples1000
            C_samples1000_60min += c_samples1000
        

In [ ]:
M_new_samples1000 = np.zeros((100,100))
C_new_samples1000 = np.zeros((100,100))
for x in range(100):
    for y in range(100):
        M_new_samples1000[x//1,y//1] += M_samples1000_60min[x,y]
        C_new_samples1000[x//1,y//1] += C_samples1000_60min[x,y]

In [ ]:
raw=(M_new_samples1000/C_new_samples1000)/(M_new_samples1000_beforeDSB/C_new_samples1000_beforeDSB)
corrected=iterative_correction_symmetric(raw)

In [ ]:
plt.imshow(np.log2(corrected[0]),cmap='cividis',vmax=0.4,vmin=-0.4)
plt.xlim([0,100])
plt.ylim([0,100])

plt.colorbar() 
plt.title("With DSB")
plt.savefig('Figures/'+'HiC_60min_2rep_1000samples_DividedByBeforeBSB_allTADs_vmax0.4_vmin-0.4_3rdParameterCombo_IterativeCorrected.pdf',format='pdf',bbox_inches='tight')

In [ ]:
base_folder_string = 'output_0310_tiling600_2populationNonBypassingLEFs_GapMethod_SparseDSB_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('.') if base_folder_string in foldername]

# Parameter combination giving more than 95% synapsis efficiency
par_combo = ['sep125_proc250_superportion0.2_superproc5000_ctcf16_dsb4_superloading1000_bs0.5']

time_point_list = [3]
box_size = 2500 # number of monomers on each side of the DSB that we will make the contact map for
short_list = []
for m in range(len(folders)):
    for n in range(len(par_combo)):
        data_dir = folders[m]
        par = par_combo[n]
        smc_file = data_dir + '/LEFcoordinates_' + par + '.npy'
        SMCs = np.load(smc_file)

        dsb_file = data_dir +'/DSB_boundary_coordinates.npy'
        with open(dsb_file, 'rb') as infile:
            DSBs = np.load(infile)
            BEs = np.load(infile)

        smcs_dict = {}
        for t in time_point_list:
            smcs_dict[t] = [(int(l),int(r)) for (l,r) in zip(SMCs[2*t,:],SMCs[2*t+1,:])]

        for d in DSBs[1::2]: # d is the right DSB end

            shift_factor = d-box_size

            for t in time_point_list: #smcs_dict.keys():
                smcs = [(x[0]-shift_factor,x[1]-shift_factor) for x in smcs_dict[t] if (np.abs(x[0]-d)<=box_size)  and (np.abs(x[1]-d)<box_size)]

                has_constraint = False
                for x in smcs:
                    if x[0]<box_size and x[1]>box_size:
                        has_constraint = True
                        break
                if has_constraint==True:
                    short_list.append(smcs) 

In [ ]:
base = 50
nreps = 2
for si, smcs in enumerate(short_list):
    dsb = [box_size-1,box_size] 
    L_tot = 2*box_size + base
    
    for rep in range(nreps):

        m_samples1000, c_samples1000 = contact_map_generator_from_SMC_list_and_DSBs(smcs,dsb,L_tot,base=base,nsamples=1000)

        if rep == 0 and si == 0:
            M_samples1000_20min = m_samples1000
            C_samples1000_20min = c_samples1000
        else:
            M_samples1000_20min += m_samples1000
            C_samples1000_20min += c_samples1000
        

In [ ]:
M_new_samples1000 = np.zeros((100,100))
C_new_samples1000 = np.zeros((100,100))
for x in range(100):
    for y in range(100):
        M_new_samples1000[x//1,y//1] += M_samples1000_20min[x,y]
        C_new_samples1000[x//1,y//1] += C_samples1000_20min[x,y]

In [ ]:
raw=(M_new_samples1000/C_new_samples1000)/(M_new_samples1000_beforeDSB/C_new_samples1000_beforeDSB)
corrected=iterative_correction_symmetric(raw)

In [ ]:
plt.imshow(np.log2(corrected[0]),cmap='cividis',vmax=0.4,vmin=-0.4)
plt.xlim([0,100])
plt.ylim([0,100])

plt.colorbar() 
plt.title("With DSB")
plt.savefig('Figures/'+'HiC_20min_2rep_1000samples_DividedByBeforeBSB_allTADs_vmax0.4_vmin-0.4_3rdParameterCombo_IterativeCorrected.pdf',format='pdf',bbox_inches='tight')

In [ ]:
base_folder_string = 'output_0310_tiling600_2populationNonBypassingLEFs_GapMethod_SparseDSB_FullParamter_DenseDSBs_run'
folders = [foldername for foldername in os.listdir('.') if base_folder_string in foldername]

# Parameter combination giving more than 95% synapsis efficiency
par_combo = ['sep125_proc250_superportion0.2_superproc5000_ctcf16_dsb4_superloading1000_bs0.5']

time_point_list = [2]
box_size = 2500 # number of monomers on each side of the DSB that we will make the contact map for
short_list = []
for m in range(len(folders)):
    for n in range(len(par_combo)):
        data_dir = folders[m]
        par = par_combo[n]
        smc_file = data_dir + '/LEFcoordinates_' + par + '.npy'
        SMCs = np.load(smc_file)

        dsb_file = data_dir +'/DSB_boundary_coordinates.npy'
        with open(dsb_file, 'rb') as infile:
            DSBs = np.load(infile)
            BEs = np.load(infile)

        smcs_dict = {}
        for t in time_point_list:
            smcs_dict[t] = [(int(l),int(r)) for (l,r) in zip(SMCs[2*t,:],SMCs[2*t+1,:])]

        for d in DSBs[1::2]: # d is the right DSB end

            shift_factor = d-box_size

            for t in time_point_list: #smcs_dict.keys():
                smcs = [(x[0]-shift_factor,x[1]-shift_factor) for x in smcs_dict[t] if (np.abs(x[0]-d)<=box_size)  and (np.abs(x[1]-d)<box_size)]

                has_constraint = False
                for x in smcs:
                    if x[0]<box_size and x[1]>box_size:
                        has_constraint = True
                        break
                if has_constraint==True:
                    short_list.append(smcs) 

In [ ]:
base = 50
nreps = 2
for si, smcs in enumerate(short_list):
    dsb = [box_size-1,box_size]
    L_tot = 2*box_size + base
    
    for rep in range(nreps):

        m_samples1000, c_samples1000 = contact_map_generator_from_SMC_list_and_DSBs(smcs,dsb,L_tot,base=base,nsamples=1000)

        if rep == 0 and si == 0:
            M_samples1000_10min = m_samples1000
            C_samples1000_10min = c_samples1000
        else:
            M_samples1000_10min += m_samples1000
            C_samples1000_10min += c_samples1000
        

In [ ]:
M_new_samples1000 = np.zeros((100,100))
C_new_samples1000 = np.zeros((100,100))
for x in range(100):
    for y in range(100):
        M_new_samples1000[x//1,y//1] += M_samples1000_10min[x,y]
        C_new_samples1000[x//1,y//1] += C_samples1000_10min[x,y]

In [ ]:
raw=(M_new_samples1000/C_new_samples1000)/(M_new_samples1000_beforeDSB/C_new_samples1000_beforeDSB)
corrected=iterative_correction_symmetric(raw)

In [ ]:
plt.imshow(np.log2(corrected[0]),cmap='cividis',vmax=0.4,vmin=-0.4)
plt.xlim([0,100])
plt.ylim([0,100])

plt.colorbar() 
plt.title("With DSB")
plt.savefig('Figures/'+'HiC_10min_2rep_1000samples_DividedByBeforeBSB_allTADs_vmax0.4_vmin-0.4_3rdParameterCombo_IterativeCorrected.pdf',format='pdf',bbox_inches='tight')